In [1]:
from google.colab import drive

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import roc_curve
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import precision_recall_curve

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

from sklearn.tree import DecisionTreeRegressor, export_graphviz

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import StackingRegressor

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from imblearn.under_sampling import RandomUnderSampler

from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

from sklearn.linear_model import LogisticRegression

from sklearn.pipeline import make_pipeline
from sklearn.svm import SVR

import xgboost

import warnings
warnings.filterwarnings('ignore')

drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Data Prepartion

In [2]:
# Load the data

path = 'drive/MyDrive/gro_homework/'

prod_qty = pd.read_csv(path+"Production Quantity.csv")
precipation = pd.read_csv(path+"Daily Precipitation.csv")
moisture = pd.read_csv(path+"Daily Soil Mositure.csv")
temperature = pd.read_csv(path+"Daily Temperature.csv")
NDVI = pd.read_csv(path+"Eight Day NDVI.csv")

pred_prod_qty = pd.read_csv(path+"predicted_production_qty.csv")


In [3]:
# Examine the 
print("Precipation:")
print(precipation.isnull().sum())

print("Moisture:")
print(moisture.isnull().sum())

print("Temperature:")
print(temperature.isnull().sum())

print("NDVI:")
print(NDVI.isnull().sum())

Precipation:
start_date    0
end_date      0
precip        0
region_id     0
dtype: int64
Moisture:
start_date    0
end_date      0
smos          0
region_id     0
dtype: int64
Temperature:
start_date    0
end_date      0
temp          0
region_id     0
dtype: int64
NDVI:
start_date    0
end_date      0
ndvi          0
region_id     0
dtype: int64


In [4]:
def date_times(df):
  df["start_date"]= pd.to_datetime(df["start_date"])
  df["end_date"]= pd.to_datetime(df["end_date"])
  # df = df[df['end_date'] >= "2014-01-01"]
  df['month_year'] = pd.to_datetime(df['start_date']).dt.to_period('M')

  # since temperature and moisture could not be accumulated
  # we would take the average value of each month in each region
  df = df.groupby(['region_id', 'month_year']).mean().reset_index()
  return df

moisture = date_times(moisture)
temperature = date_times(temperature)
precipation = date_times(precipation)
NDVI = date_times(NDVI)

In [5]:
new_df = pd.merge(NDVI, 
                  moisture,  
                  how='inner', 
                  left_on=['month_year','region_id'], 
                  right_on = ['month_year','region_id'])

new_df = pd.merge(new_df, 
                  temperature,  
                  how='inner', 
                  left_on=['month_year','region_id'], 
                  right_on = ['month_year','region_id'])

new_df = pd.merge(new_df, 
                  precipation,  
                  how='inner', 
                  left_on=['month_year','region_id'], 
                  right_on = ['month_year','region_id'])

In [6]:
def date_times(df):
  df["start_date"]= pd.to_datetime(df["start_date"])
  df["end_date"]= pd.to_datetime(df["end_date"])
  # df = df[df['end_date'] >= "2014-01-01"]
  df['month_year'] = pd.to_datetime(df['end_date']).dt.to_period('M')

  df = df.drop(columns = ["start_date", "end_date"])

  return df

prod_qty = date_times(prod_qty)
target_qty = date_times(pred_prod_qty)


In [7]:
developing_df = pd.merge(new_df, 
                       prod_qty,  
                       how='inner', 
                       left_on=['month_year','region_id'], 
                       right_on = ['month_year','region_id']).drop(columns = ['month_year'])

testing_df = pd.merge(new_df, 
                       target_qty ,  
                       how='inner', 
                       left_on=['month_year','region_id'], 
                       right_on = ['month_year','region_id']).drop(columns = ['month_year', 'prod'])

# Use one-hot encoding to encode the variable region id

developing_df = pd.get_dummies(developing_df, 
                             columns=["region_id"], 
                             prefix=["Region_"])

testing_df = pd.get_dummies(testing_df, 
                             columns=["region_id"], 
                             prefix=["Region_"])

In [8]:
X_dev = developing_df.loc[:, developing_df.columns != 'prod']
y_dev = developing_df.loc[:, developing_df.columns == 'prod']

# train test split
X_train, X_test, y_train, y_test = train_test_split(X_dev, 
                                                    y_dev, 
                                                    test_size=0.2,  
                                                    # 0.8 dev data vs 0.2 test data
                                                    random_state=123)

In [9]:
#list for cols to scale
cols_to_scale = ['ndvi', 'smos', 'precip','temp']

#create and fit scaler
scaler = StandardScaler()
scaler.fit(X_train[cols_to_scale])

#scale selected data
X_train[cols_to_scale] = scaler.transform(X_train[cols_to_scale])
X_test[cols_to_scale] = scaler.transform(X_test[cols_to_scale])

## Fit with Supervised Learning Models

In [10]:
# Start with the fundamental decision treemodel

tree_para = {'max_depth':[4,5,6,7,8,9,10,12,15,20],
             'max_features':[3, 4, 5, 6]}

clf = GridSearchCV(DecisionTreeRegressor(random_state = 123), 
                   tree_para, 
                   cv=5)

clf.fit(X_train, y_train)

print(f"The optimal parameter is   : {clf.best_params_}")
print(f"The best training score is : {clf.best_score_}")
print(f"The best testing score is  : {clf.best_estimator_.score(X_test, y_test)}")

The optimal parameter is   : {'max_depth': 5, 'max_features': 4}
The best training score is : 0.9092235394029637
The best testing score is  : 0.9341128011368088


In [11]:
# Then try the SVM method

param_grid = {'C': [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
              'kernel': ['rbf', 'ploy','linear']}
 
clf = GridSearchCV(SVR(), param_grid, cv=5)
 
# fitting the model for grid search
clf.fit(X_train, y_train)

print(f"The optimal parameter is   : {clf.best_params_}")
print(f"The best training score is : {clf.best_score_}")
print(f"The best testing score is  : {clf.best_estimator_.score(X_test, y_test)}")


The optimal parameter is   : {'C': 100, 'kernel': 'linear'}
The best training score is : 0.07034246421779747
The best testing score is  : 0.18334847731416593


In [12]:
tree_para = {'max_depth':[4,5,6,7,8,9,10,12,15,20,25,50],
             'max_features':['auto', 'sqrt', 'log2']
             }

clf = GridSearchCV(RandomForestRegressor(random_state = 123), 
                   tree_para, 
                   cv=5)

clf.fit(X_train, y_train)

print(f"The optimal parameter is   : {clf.best_params_}")
print(f"The best training score is : {clf.best_score_}")
print(f"The best testing score is  : {clf.best_estimator_.score(X_test, y_test)}")

The optimal parameter is   : {'max_depth': 20, 'max_features': 'auto'}
The best training score is : 0.9374482818308856
The best testing score is  : 0.9603912745608829


In [13]:
tree_para = {'loss':['squared_error', 'absolute_error', 'huber', 'quantile'],
             'n_estimators':[100,150,200],
             # 'learning_rate':[0.001, 0.005, 0.01, 0.02, 0.05, 0.1],
             'max_depth':[10,12,15,20,25,50],
             # 'max_features':['auto', 'sqrt', 'log2']
             }

clf = GridSearchCV(GradientBoostingRegressor(random_state = 123), 
                   tree_para, 
                   cv=5)

clf.fit(X_train, y_train)

print(f"The optimal parameter is   : {clf.best_params_}")
print(f"The best training score is : {clf.best_score_}")
print(f"The best testing score is  : {clf.best_estimator_.score(X_test, y_test)}")

The optimal parameter is   : {'loss': 'absolute_error', 'max_depth': 20, 'n_estimators': 150}
The best training score is : 0.932865272854408
The best testing score is  : 0.9690757140258924


In [ ]:
tree_para = {'n_estimators':[100, 150, 200, 250], 
             'max_depth':[10,12,15,20,25], 
             'eta':[0.01, 0.02, 0.05, 0.1]
             }

clf = GridSearchCV(xgboost.XGBRegressor(random_state = 123,
                                        verbosity = 0), 
                   tree_para, 
                   cv=5)

clf.fit(X_train, y_train)

print(f"The optimal parameter is   : {clf.best_params_}")
print(f"The best training score is : {clf.best_score_}")
print(f"The best testing score is  : {clf.best_estimator_.score(X_test, y_test)}")

In [14]:
# create a staking 

estimators = [('Rf', RandomForestRegressor(random_state = 123,
                                           max_depth = 20, 
                                           max_features = 'auto')),
              ('GBR', GradientBoostingRegressor(random_state = 123, 
                                                loss ='absolute_error', 
                                                max_depth = 20, 
                                                n_estimators = 150)),
              ('XGB', xgboost.XGBRegressor(random_state = 123,
                                           verbosity = 0,
                                           eta = 0.01, 
                                           max_depth= 10, 
                                           n_estimators = 100)),
              ('DT', DecisionTreeRegressor(random_state = 123, 
                                           max_depth= 5, 
                                           max_features= 4))
              ]

reg = StackingRegressor(estimators=estimators, final_estimator=xgboost.XGBRegressor(random_state = 123,
                                           verbosity = 0,
                                           eta = 0.01, 
                                           max_depth= 10, 
                                           n_estimators = 100)
                        )

print(f"The testing accuracy: {reg.fit(X_train, y_train).score(X_test, y_test)}")


The testing accuracy: 0.9449257968638008


In [31]:
pred_prod_qty['prod'] = reg.predict(testing_df).round().astype(int)

pred_prod_qty = pred_prod_qty.drop(columns = ['month_year'])

pred_prod_qty.to_csv(path+'cx2274@columbia.edu.csv',index=False)  